In [ ]:
#import os

# args.output_attention = True

#exp = Exp(args)

#model = exp.model

#setting = 'ETTh1_96_24_Autoformer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0'
#path = os.path.join(args.checkpoints,setting,'checkpoint.pth')
#model.load_state_dict(torch.load(path))

In [ ]:
import torch
import os
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
print(torch.cuda.current_device())
torch.cuda.set_device(0)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = '0' #disable CUDA by setting CUDA_VISIBLE_DEVICES to an empty string 


In finance and stock market analysis, it's not uncommon to consider a small percentage change as "neutral" or "no change" because daily stock prices can fluctuate due to a variety of factors, including market volatility. By setting a threshold of 0.25%, you're essentially saying that any change in price less than this percentage is insignificant and should be considered neutral. 

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

commodities = ['gold', 'silver', 'natural_gas', 'coffee', 'corn', 'crude_oil']
OT_dict = {0: 'Decrease', 1: 'Increase'}

fig, axes = plt.subplots(nrows=1, ncols=len(commodities), figsize=(20, 5))


for i, commodity in enumerate(commodities):
    df = pd.read_csv(f'./dataset/commodity/{commodity}_data.csv')

    df['date'] = pd.to_datetime(df['date'])

    # Make sure the 'close' column is correctly named
    df.columns = df.columns.str.replace('OT', 'close')

    # Calculate the percentage change between the closing price of the next day and the current day
    df['close_pct_change'] = df['close'].pct_change(periods=-1) * 100
    
    # Create the target column based on the 'close_pct_change' column
    df['OT'] = 0.0 # neutral
    df.loc[df['close_pct_change'] > 0, 'OT'] = 1.0 # increase
    # df.loc[df['close_pct_change'] < -0.25, 'OT'] = 2 # decrease

    df = df.drop(columns=['close_pct_change'])
    df = df.drop(columns=['close', 'Adj Close'])

    # Drop the last row as it doesn't have a valid 'OT' value
    df = df[:-1]

    # Save the dataframe as a new CSV file
    df.to_csv(f'./dataset/commodity/{commodity}_data_c.csv')
    
    sns.countplot(x='OT', data=df, ax=axes[i])
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels([OT_dict[i] for i in [0, 1]])
    axes[i].set_title(f'Distribution of {commodity} classes')

# Displaying the plots
plt.tight_layout()
plt.savefig('classification_bar.png')
plt.show()

In [ ]:
import pandas as pd
df = pd.read_csv(f'./dataset/commodity/gold_data_c.csv')
df.info()

In [1]:
# D Linear

import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

models = ['Autoformer', 'Informer', 'Transformer', 'DLinear', 'NLinear']
data_paths = ['gold_data_c.csv', 'silver_data_c.csv', 'crude_oil_data_c.csv', 'natural_gas_data_c.csv', 'corn_data_c.csv', 'coffee_data_c.csv']
label_lens = [14, 28, 42, 56]
pred_lens = [56]
args = dotdict()

for data_path in data_paths:
    for pred_len in pred_lens:

        # basic config
        args.is_training = 1
        args.model_id = ''
        args.model = 'DLinear' #Autoformer/Informer/Transformer/DLinear/NLinear

        # data loader
        args.data = 'custom'
        args.root_path = './dataset/commodity/'
        args.data_path = data_path
        args.features = 'M'
        args.target = 'OT'
        args.freq = 'd'
        args.checkpoints = './checkpoints/'

        # forecasting task
        args.seq_len = 96
        args.label_len = pred_len
        args.pred_len = pred_len

        # DLinear
        args.individual = False

        # Formers 
        args.embed_type = 0
        args.enc_in = 6
        args.dec_in = 6
        args.c_out = 6
        args.d_model = 512
        args.n_heads = 8
        args.e_layers = 2
        args.d_layers = 1
        args.d_ff = 2048
        args.moving_avg = 25
        args.factor = 3
        args.distil = True
        args.dropout = 0.05
        args.embed = 'timeF'
        args.do_predict = True

        # optimization
        args.num_workers = 10
        args.itr = 1
        args.train_epochs = 50
        args.patience = 5
        args.learning_rate = 0.0005
        args.batch_size = 8
        args.lradj = 'type1'
        args.des = 'Exp'

        # GPU
        args.gpu = 0
        args.devices = '0'
        args.use_gpu = True
        args.use_multi_gpu = False

        # print('Args in experiment:')
        # print(args)

        Exp = Exp_Main
        print(f'######################  {data_path}_{args.model}_{pred_len}_{pred_len}  ######################')
        if args.is_training:
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                    args.data_path.split("_")[0],
                    args.model_id,
                    args.model,
                    args.data,
                    args.features,
                    args.seq_len,
                    args.label_len,
                    args.pred_len,
                    args.d_model,
                    args.n_heads,
                    args.e_layers,
                    args.d_layers,
                    args.d_ff,
                    args.factor,
                    args.embed,
                    args.distil,
                    args.des, ii)

                exp = Exp(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                exp.test(setting)

                torch.cuda.empty_cache()


######################  gold_data_c.csv_DLinear_56_56  ######################
Use GPU: cuda:0
>>>>>>>start training : gold__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1608
val 197
test 447
	iters: 100, epoch: 1 | loss: 0.4515233
	speed: 0.2572s/iter; left time: 2559.4394s
	iters: 200, epoch: 1 | loss: 0.5054529
	speed: 0.0026s/iter; left time: 25.7438s
Epoch: 1 cost time: 26.88035297393799
Epoch: 1, Steps: 201 | Train Loss: 0.4764939 Vali Loss: 1.4964520 Test Loss: 0.9893274
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.4311605
	speed: 0.3708s/iter; left time: 3615.2854s
	iters: 200, epoch: 2 | loss: 0.5266655
	speed: 0.0022s/iter; left time: 21.6547s
Epoch: 2 cost time: 24.580622673034668
Epoch: 2, Steps: 201 | Train Loss: 0.4432168 Vali Loss: 1.3520074 Test Loss: 0.8852887
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.4864195
	speed: 0.3779s/iter; left time: 3608.8218s
	ite

accuracy, f1: 0.744 & 0.709 
440, 440
######################  crude_oil_data_c.csv_DLinear_56_56  ######################
Use GPU: cuda:0
>>>>>>>start training : crude__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1609
val 197
test 448
	iters: 100, epoch: 1 | loss: 0.3636216
	speed: 0.2346s/iter; left time: 2334.2418s
	iters: 200, epoch: 1 | loss: 0.4252815
	speed: 0.0023s/iter; left time: 22.2039s
Epoch: 1 cost time: 24.502159595489502
Epoch: 1, Steps: 201 | Train Loss: 0.3394222 Vali Loss: 0.7322459 Test Loss: 0.3920832
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.4000925
	speed: 0.3790s/iter; left time: 3694.8723s
	iters: 200, epoch: 2 | loss: 0.3211789
	speed: 0.0022s/iter; left time: 21.1756s
Epoch: 2 cost time: 24.31145691871643
Epoch: 2, Steps: 201 | Train Loss: 0.3158242 Vali Loss: 0.7225709 Test Loss: 0.3844586
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.2705851
	spe

Epoch: 8 cost time: 26.886470317840576
Epoch: 8, Steps: 201 | Train Loss: 0.4725164 Vali Loss: 0.5384447 Test Loss: 1.9978476
EarlyStopping counter: 4 out of 5
Updating learning rate to 3.90625e-06
	iters: 100, epoch: 9 | loss: 0.4365718
	speed: 0.3869s/iter; left time: 3227.5665s
	iters: 200, epoch: 9 | loss: 0.5031409
	speed: 0.0022s/iter; left time: 17.7573s
Epoch: 9 cost time: 24.930137872695923
Epoch: 9, Steps: 201 | Train Loss: 0.4723713 Vali Loss: 0.5414367 Test Loss: 1.9970576
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : natural__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 448
accuracy, f1: 0.720 & 0.685 
448, 448
######################  corn_data_c.csv_DLinear_56_56  ######################
Use GPU: cuda:0
>>>>>>>start training : corn__DLinear_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1606
val 197
test 447
	iters: 1

In [6]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

models = ['Autoformer', 'Informer', 'Transformer', 'DLinear', 'NLinear']
data_paths = ['gold_data_c.csv', 'silver_data_c.csv', 'crude_oil_data_c.csv', 'natural_gas_data_c.csv', 'corn_data_c.csv', 'coffee_data_c.csv']
label_lens = [14, 28, 42, 56]
pred_lens = [14, 28, 42, 56]
args = dotdict()

for data_path in data_paths:
    for pred_len in pred_lens:

        # basic config
        args.is_training = 1
        args.model_id = 'classification'
        args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

        # data loader
        args.data = 'custom'
        args.root_path = './dataset/commodity/'
        args.data_path = data_path
        args.features = 'M'
        args.target = 'OT'
        args.freq = 'd'
        args.checkpoints = './checkpoints/'

        # forecasting task
        args.seq_len = 96
        args.label_len = pred_len
        args.pred_len = pred_len

        # DLinear
        args.individual = False

        # Formers 
        args.embed_type = 0
        args.enc_in = 6
        args.dec_in = 6
        args.c_out = 6
        args.d_model = 512
        args.n_heads = 8
        args.e_layers = 2
        args.d_layers = 1
        args.d_ff = 2048
        args.moving_avg = 25
        args.factor = 3
        args.distil = True
        args.dropout = 0.05
        args.embed = 'timeF'
        args.do_predict = True

        # optimization
        args.num_workers = 10
        args.itr = 1
        args.train_epochs = 50
        args.patience = 5
        args.learning_rate = 0.0005
        args.batch_size = 8
        args.lradj = 'type1'
        args.des = 'Exp'

        # GPU
        args.gpu = 0
        args.devices = '0'
        args.use_gpu = True
        args.use_multi_gpu = False

        # print('Args in experiment:')
        # print(args)

        Exp = Exp_Main
        print(f'######################  {data_path}_{args.model}_{pred_len}_{pred_len}  ######################')
        if args.is_training:
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                    args.data_path.split("_")[0],
                    args.model_id,
                    args.model,
                    args.data,
                    args.features,
                    args.seq_len,
                    args.label_len,
                    args.pred_len,
                    args.d_model,
                    args.n_heads,
                    args.e_layers,
                    args.d_layers,
                    args.d_ff,
                    args.factor,
                    args.embed,
                    args.distil,
                    args.des, ii)

                exp = Exp(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                exp.test(setting)

                torch.cuda.empty_cache()


######################  gold_data_c.csv_Autoformer_14_14  ######################
Use GPU: cuda:0
>>>>>>>start training : gold_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1650
val 239
test 489
	iters: 100, epoch: 1 | loss: 0.7150675
	speed: 0.2876s/iter; left time: 2933.9672s
	iters: 200, epoch: 1 | loss: 1.0009717
	speed: 0.0523s/iter; left time: 527.7991s
Epoch: 1 cost time: 35.08281636238098
Epoch: 1, Steps: 206 | Train Loss: 0.7102846 Vali Loss: 5.8019609 Test Loss: 8.5066147
Validation loss decreased (inf --> 5.801961).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.6824962
	speed: 0.4688s/iter; left time: 4685.9951s
	iters: 200, epoch: 2 | loss: 0.5746744
	speed: 0.0523s/iter; left time: 517.6164s
Epoch: 2 cost time: 35.66663384437561
Epoch: 2, Steps: 206 | Train Loss: 0.7020987 Vali Loss: 5.6443119 Test Loss: 8.4638128
Validation loss decreased (5.801961 -

	iters: 100, epoch: 14 | loss: 0.5757647
	speed: 0.5506s/iter; left time: 4101.3575s
	iters: 200, epoch: 14 | loss: 0.5699505
	speed: 0.0601s/iter; left time: 441.7375s
Epoch: 14 cost time: 40.53955602645874
Epoch: 14, Steps: 204 | Train Loss: 0.6461241 Vali Loss: 5.6951518 Test Loss: 8.5104895
EarlyStopping counter: 4 out of 5
Updating learning rate to 6.103515625e-08
	iters: 100, epoch: 15 | loss: 0.6900135
	speed: 0.5100s/iter; left time: 3695.2613s
	iters: 200, epoch: 15 | loss: 0.7408783
	speed: 0.0620s/iter; left time: 443.3261s
Epoch: 15 cost time: 38.941726207733154
Epoch: 15, Steps: 204 | Train Loss: 0.6469281 Vali Loss: 5.6701417 Test Loss: 8.5104685
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : gold_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 475
( accuracy, f1: 0.690 & 0.711 
472, 472
79296, 79296
 accuracy, f1: 0.273 & 0.150 
######################  gold_data

	iters: 200, epoch: 7 | loss: 0.6741790
	speed: 0.0671s/iter; left time: 579.9081s
Epoch: 7 cost time: 41.78963088989258
Epoch: 7, Steps: 201 | Train Loss: 0.5651924 Vali Loss: 5.8021598 Test Loss: 8.7012186
EarlyStopping counter: 4 out of 5
Updating learning rate to 7.8125e-06
	iters: 100, epoch: 8 | loss: 0.6579133
	speed: 0.5444s/iter; left time: 4651.3574s
	iters: 200, epoch: 8 | loss: 0.5703865
	speed: 0.0679s/iter; left time: 573.4148s
Epoch: 8 cost time: 41.866419076919556
Epoch: 8, Steps: 201 | Train Loss: 0.5639870 Vali Loss: 5.8640161 Test Loss: 8.6802149
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : gold_classification_Autoformer_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 447
( accuracy, f1: 0.708 & 0.742 
440, 440
147840, 147840
 accuracy, f1: 0.275 & 0.159 
######################  silver_data_c.csv_Autoformer_14_14  ######################
Use GPU: cuda:0
>>>>>>>start training : silve

	iters: 200, epoch: 1 | loss: 0.5427881
	speed: 0.0643s/iter; left time: 636.5045s
Epoch: 1 cost time: 41.497105836868286
Epoch: 1, Steps: 202 | Train Loss: 0.6104436 Vali Loss: 1.0630281 Test Loss: 2.0787911
Validation loss decreased (inf --> 1.063028).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.7407855
	speed: 0.5492s/iter; left time: 5381.2919s
	iters: 200, epoch: 2 | loss: 0.6384847
	speed: 0.0648s/iter; left time: 628.3207s
Epoch: 2 cost time: 41.544090270996094
Epoch: 2, Steps: 202 | Train Loss: 0.5916304 Vali Loss: 1.0803471 Test Loss: 2.0822527
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.5769884
	speed: 0.5467s/iter; left time: 5246.2958s
	iters: 200, epoch: 3 | loss: 0.5404185
	speed: 0.0646s/iter; left time: 613.5083s
Epoch: 3 cost time: 41.4304575920105
Epoch: 3, Steps: 202 | Train Loss: 0.5744320 Vali Loss: 1.0797876 Test Loss: 2.1002657
EarlyStopping counter: 2 out of 5
Updating

Epoch: 2 cost time: 41.06181979179382
Epoch: 2, Steps: 200 | Train Loss: 0.5851395 Vali Loss: 0.9833139 Test Loss: 2.0300248
Validation loss decreased (1.000865 --> 0.983314).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.5657621
	speed: 0.5269s/iter; left time: 5005.6086s
	iters: 200, epoch: 3 | loss: 0.4898215
	speed: 0.0675s/iter; left time: 634.4131s
Epoch: 3 cost time: 40.61103701591492
Epoch: 3, Steps: 200 | Train Loss: 0.5616923 Vali Loss: 1.0702858 Test Loss: 2.2037942
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.000125
	iters: 100, epoch: 4 | loss: 0.6456875
	speed: 0.5281s/iter; left time: 4911.5509s
	iters: 200, epoch: 4 | loss: 0.5940708
	speed: 0.0675s/iter; left time: 621.3455s
Epoch: 4 cost time: 40.85618448257446
Epoch: 4, Steps: 200 | Train Loss: 0.5463177 Vali Loss: 1.1374229 Test Loss: 2.2596152
EarlyStopping counter: 2 out of 5
Updating learning rate to 6.25e-05
	iters: 100, epoch: 5 | loss: 0.5227545
	speed: 0.

...

	iters: 200, epoch: 8 | loss: 0.8025526
	speed: 0.1455s/iter; left time: 1228.6785s
Epoch: 8 cost time: 86.20182943344116
Epoch: 8, Steps: 201 | Train Loss: 0.7600442 Vali Loss: 1.5473284 Test Loss: 1.4566231
EarlyStopping counter: 4 out of 5
Updating learning rate to 3.90625e-06
	iters: 100, epoch: 9 | loss: 0.5957254
	speed: 1.0668s/iter; left time: 8900.7088s
	iters: 200, epoch: 9 | loss: 0.6889934
	speed: 0.1440s/iter; left time: 1186.9586s
Epoch: 9 cost time: 85.6065788269043
Epoch: 9, Steps: 201 | Train Loss: 0.7594314 Vali Loss: 1.5372337 Test Loss: 1.4537144
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : crude_classification_Autoformer_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 448
( accuracy, f1: 0.735 & 0.755 
448, 448
150528, 150528
 accuracy, f1: 0.525 & 0.477 
######################  natural_gas_data_c.csv_Autoformer_14_14  ######################
Use GPU: cuda:0
>>>>>>>start training 

	iters: 200, epoch: 4 | loss: 1.1013256
	speed: 0.1397s/iter; left time: 1298.5112s
Epoch: 4 cost time: 81.91245675086975
Epoch: 4, Steps: 202 | Train Loss: 0.8565149 Vali Loss: 2.1160595 Test Loss: 6.9308090
EarlyStopping counter: 2 out of 5
Updating learning rate to 6.25e-05
	iters: 100, epoch: 5 | loss: 0.6474276
	speed: 1.0664s/iter; left time: 9803.0880s
	iters: 200, epoch: 5 | loss: 1.0497553
	speed: 0.1383s/iter; left time: 1257.6325s
Epoch: 5 cost time: 83.5029788017273
Epoch: 5, Steps: 202 | Train Loss: 0.8510066 Vali Loss: 2.1033309 Test Loss: 6.9443541
Validation loss decreased (2.115107 --> 2.103331).  Saving model ...
Updating learning rate to 3.125e-05
	iters: 100, epoch: 6 | loss: 0.7606266
	speed: 1.1204s/iter; left time: 10073.5949s
	iters: 200, epoch: 6 | loss: 1.1932918
	speed: 0.1475s/iter; left time: 1311.6062s
Epoch: 6 cost time: 85.14688634872437
Epoch: 6, Steps: 202 | Train Loss: 0.8437252 Vali Loss: 2.1208043 Test Loss: 6.9368019
EarlyStopping counter: 1 out of

Epoch: 4 cost time: 85.26020169258118
Epoch: 4, Steps: 206 | Train Loss: 0.6541644 Vali Loss: 0.7800537 Test Loss: 3.0948226
Validation loss decreased (0.789644 --> 0.780054).  Saving model ...
Updating learning rate to 6.25e-05
	iters: 100, epoch: 5 | loss: 0.7420205
	speed: 1.1314s/iter; left time: 10609.4470s
	iters: 200, epoch: 5 | loss: 0.5713568
	speed: 0.1343s/iter; left time: 1245.8005s
Epoch: 5 cost time: 88.55787682533264
Epoch: 5, Steps: 206 | Train Loss: 0.6479821 Vali Loss: 0.7671589 Test Loss: 3.0893984
Validation loss decreased (0.780054 --> 0.767159).  Saving model ...
Updating learning rate to 3.125e-05
	iters: 100, epoch: 6 | loss: 0.4323314
	speed: 1.0755s/iter; left time: 9863.1836s
	iters: 200, epoch: 6 | loss: 0.8161846
	speed: 0.1323s/iter; left time: 1199.9450s
Epoch: 6 cost time: 83.43831515312195
Epoch: 6, Steps: 206 | Train Loss: 0.6458276 Vali Loss: 0.7821642 Test Loss: 3.1041033
EarlyStopping counter: 1 out of 5
Updating learning rate to 1.5625e-05
	iters: 

	iters: 100, epoch: 7 | loss: 0.5220477
	speed: 1.1232s/iter; left time: 9871.9541s
	iters: 200, epoch: 7 | loss: 0.4965756
	speed: 0.1426s/iter; left time: 1239.2999s
Epoch: 7 cost time: 87.72499418258667
Epoch: 7, Steps: 202 | Train Loss: 0.6130215 Vali Loss: 0.8281145 Test Loss: 3.1801648
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : corn_classification_Autoformer_custom_ftM_sl96_ll42_pl42_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 461
( accuracy, f1: 0.710 & 0.740 
456, 456
114912, 114912
 accuracy, f1: 0.309 & 0.268 
######################  corn_data_c.csv_Autoformer_14_56  ######################
Use GPU: cuda:0
>>>>>>>start training : corn_classification_Autoformer_custom_ftM_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1606
val 197
test 447
	iters: 100, epoch: 1 | loss: 0.7646302
	speed: 0.7183s/iter; left time: 7112.0518s
	iters: 200, epoch: 1 | loss: 0.5589564
	

	iters: 100, epoch: 5 | loss: 0.8800078
	speed: 1.0600s/iter; left time: 9841.9165s
	iters: 200, epoch: 5 | loss: 0.7951154
	speed: 0.1330s/iter; left time: 1221.2961s
Epoch: 5 cost time: 82.7793653011322
Epoch: 5, Steps: 204 | Train Loss: 0.8450002 Vali Loss: 1.0224820 Test Loss: 4.3075194
EarlyStopping counter: 3 out of 5
Updating learning rate to 3.125e-05
	iters: 100, epoch: 6 | loss: 0.8083376
	speed: 1.0829s/iter; left time: 9834.1000s
	iters: 200, epoch: 6 | loss: 0.8800920
	speed: 0.1359s/iter; left time: 1220.8867s
Epoch: 6 cost time: 84.97136664390564
Epoch: 6, Steps: 204 | Train Loss: 0.8415262 Vali Loss: 1.0162466 Test Loss: 4.3193998
EarlyStopping counter: 4 out of 5
Updating learning rate to 1.5625e-05
	iters: 100, epoch: 7 | loss: 0.7635168
	speed: 1.0770s/iter; left time: 9560.7007s
	iters: 200, epoch: 7 | loss: 0.9790399
	speed: 0.1520s/iter; left time: 1334.5310s
Epoch: 7 cost time: 85.37327241897583
Epoch: 7, Steps: 204 | Train Loss: 0.8394268 Vali Loss: 1.0295898 Te

In [1]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

models = ['Autoformer', 'Informer', 'Transformer', 'DLinear', 'NLinear']
data_paths = ['gold_data_c.csv', 'silver_data_c.csv', 'crude_oil_data_c.csv', 'natural_gas_data_c.csv', 'corn_data_c.csv', 'coffee_data_c.csv']
label_lens = [14, 28, 42, 56]
pred_lens = [14, 28, 42, 56]
args = dotdict()

for data_path in data_paths:
    for pred_len in pred_lens:

        # basic config
        args.is_training = 1
        args.model_id = 'classification'
        args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

        # data loader
        args.data = 'custom'
        args.root_path = './dataset/commodity/'
        args.data_path = data_path
        args.features = 'S'
        args.target = 'OT'
        args.freq = 'd'
        args.checkpoints = './checkpoints/'

        # forecasting task
        args.seq_len = 96
        args.label_len = pred_len
        args.pred_len = pred_len

        # DLinear
        args.individual = False

        # Formers 
        args.embed_type = 0
        args.enc_in = 1
        args.dec_in = 1
        args.c_out = 2
        args.d_model = 512
        args.n_heads = 8
        args.e_layers = 2
        args.d_layers = 1
        args.d_ff = 2048
        args.moving_avg = 25
        args.factor = 3
        args.distil = True
        args.dropout = 0.05
        args.embed = 'timeF'
        args.do_predict = True

        # optimization
        args.num_workers = 10
        args.itr = 1
        args.train_epochs = 50
        args.patience = 5
        args.learning_rate = 0.0005
        args.batch_size = 8
        args.lradj = 'type1'
        args.des = 'Exp'

        # GPU
        args.gpu = 0
        args.devices = '0'
        args.use_gpu = True
        args.use_multi_gpu = False

        # print('Args in experiment:')
        # print(args)

        Exp = Exp_Main
        print(f'######################  {data_path}_{args.model}_{pred_len}_{pred_len}  ######################')
        if args.is_training:
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                    args.data_path.split("_")[0],
                    args.model_id,
                    args.model,
                    args.data,
                    args.features,
                    args.seq_len,
                    args.label_len,
                    args.pred_len,
                    args.d_model,
                    args.n_heads,
                    args.e_layers,
                    args.d_layers,
                    args.d_ff,
                    args.factor,
                    args.embed,
                    args.distil,
                    args.des, ii)

                exp = Exp(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                exp.test(setting)

                torch.cuda.empty_cache()


######################  gold_data_c.csv_Autoformer_14_14  ######################
Use GPU: cuda:0
>>>>>>>start training : gold_classification_Autoformer_custom_ftS_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1650
val 239
test 489
	iters: 100, epoch: 1 | loss: 1.1885455
	speed: 0.1483s/iter; left time: 1513.2607s
	iters: 200, epoch: 1 | loss: 1.1154827
	speed: 0.0522s/iter; left time: 526.9359s
Epoch: 1 cost time: 21.398499965667725
Epoch: 1, Steps: 206 | Train Loss: 1.2541343 Vali Loss: 1.4313439 Test Loss: 1.3314521
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 1.4439062
	speed: 0.3182s/iter; left time: 3180.6061s
	iters: 200, epoch: 2 | loss: 1.1518065
	speed: 0.0575s/iter; left time: 569.3947s
Epoch: 2 cost time: 18.805977821350098
Epoch: 2, Steps: 206 | Train Loss: 1.2498268 Vali Loss: 1.4357511 Test Loss: 1.3314862
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 1

	iters: 200, epoch: 9 | loss: 1.2154955
	speed: 0.0580s/iter; left time: 485.2764s
Epoch: 9 cost time: 19.14796018600464
Epoch: 9, Steps: 204 | Train Loss: 1.2493841 Vali Loss: 1.4418538 Test Loss: 1.3299989
EarlyStopping counter: 3 out of 5
Updating learning rate to 1.953125e-06
	iters: 100, epoch: 10 | loss: 1.2154937
	speed: 0.3161s/iter; left time: 2612.9649s
	iters: 200, epoch: 10 | loss: 1.3431932
	speed: 0.0581s/iter; left time: 474.1312s
Epoch: 10 cost time: 19.135037660598755
Epoch: 10, Steps: 204 | Train Loss: 1.2492937 Vali Loss: 1.4421794 Test Loss: 1.3299983
EarlyStopping counter: 4 out of 5
Updating learning rate to 9.765625e-07
	iters: 100, epoch: 11 | loss: 1.3796924
	speed: 0.3142s/iter; left time: 2532.7286s
	iters: 200, epoch: 11 | loss: 1.3431909
	speed: 0.0578s/iter; left time: 460.2885s
Epoch: 11 cost time: 19.048155069351196
Epoch: 11, Steps: 204 | Train Loss: 1.2496076 Vali Loss: 1.4425050 Test Loss: 1.3299981
EarlyStopping counter: 5 out of 5
Early stopping
>>>

Epoch: 2 cost time: 18.46051335334778
Epoch: 2, Steps: 206 | Train Loss: 1.2471112 Vali Loss: 1.3566152 Test Loss: 1.2831415
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 1.2895266
	speed: 0.3124s/iter; left time: 3057.6536s
	iters: 200, epoch: 3 | loss: 1.2715768
	speed: 0.0539s/iter; left time: 522.6767s
Epoch: 3 cost time: 18.463436365127563
Epoch: 3, Steps: 206 | Train Loss: 1.2467942 Vali Loss: 1.3619884 Test Loss: 1.2829658
EarlyStopping counter: 2 out of 5
Updating learning rate to 0.000125
	iters: 100, epoch: 4 | loss: 1.2535319
	speed: 0.3123s/iter; left time: 2992.8935s
	iters: 200, epoch: 4 | loss: 1.3074936
	speed: 0.0538s/iter; left time: 510.5664s
Epoch: 4 cost time: 18.46105456352234
Epoch: 4, Steps: 206 | Train Loss: 1.2463994 Vali Loss: 1.3607258 Test Loss: 1.2829438
EarlyStopping counter: 3 out of 5
Updating learning rate to 6.25e-05
	iters: 100, epoch: 5 | loss: 1.2356154
	speed: 0.3100s/iter; left time: 2907.1586s


	iters: 200, epoch: 11 | loss: 1.1516013
	speed: 0.0623s/iter; left time: 491.3727s
Epoch: 11 cost time: 19.880714893341064
Epoch: 11, Steps: 202 | Train Loss: 1.2507270 Vali Loss: 1.3712285 Test Loss: 1.2833022
EarlyStopping counter: 1 out of 5
Updating learning rate to 4.8828125e-07
	iters: 100, epoch: 12 | loss: 1.2115302
	speed: 0.3194s/iter; left time: 2484.3669s
	iters: 200, epoch: 12 | loss: 1.3253894
	speed: 0.0623s/iter; left time: 478.5355s
Epoch: 12 cost time: 19.905675888061523
Epoch: 12, Steps: 202 | Train Loss: 1.2506082 Vali Loss: 1.3714588 Test Loss: 1.2833019
EarlyStopping counter: 2 out of 5
Updating learning rate to 2.44140625e-07
	iters: 100, epoch: 13 | loss: 1.2474918
	speed: 0.3196s/iter; left time: 2421.3871s
	iters: 200, epoch: 13 | loss: 1.3254161
	speed: 0.0623s/iter; left time: 466.1447s
Epoch: 13 cost time: 19.849976062774658
Epoch: 13, Steps: 202 | Train Loss: 1.2508457 Vali Loss: 1.3700758 Test Loss: 1.2833021
Updating learning rate to 1.220703125e-07
	it

Epoch: 7 cost time: 18.545409440994263
Epoch: 7, Steps: 206 | Train Loss: 1.2492098 Vali Loss: 1.2761319 Test Loss: 1.3861380
Updating learning rate to 7.8125e-06
	iters: 100, epoch: 8 | loss: 1.2213056
	speed: 0.3156s/iter; left time: 2764.2800s
	iters: 200, epoch: 8 | loss: 1.2044894
	speed: 0.0539s/iter; left time: 466.9292s
Epoch: 8 cost time: 18.54739284515381
Epoch: 8, Steps: 206 | Train Loss: 1.2491285 Vali Loss: 1.2830718 Test Loss: 1.3861393
EarlyStopping counter: 1 out of 5
Updating learning rate to 3.90625e-06
	iters: 100, epoch: 9 | loss: 1.3219464
	speed: 0.3145s/iter; left time: 2689.7348s
	iters: 200, epoch: 9 | loss: 1.3386514
	speed: 0.0540s/iter; left time: 456.5714s
Epoch: 9 cost time: 18.557457447052002
Epoch: 9, Steps: 206 | Train Loss: 1.2494500 Vali Loss: 1.2836471 Test Loss: 1.3861362
EarlyStopping counter: 2 out of 5
Updating learning rate to 1.953125e-06
	iters: 100, epoch: 10 | loss: 1.2883538
	speed: 0.3122s/iter; left time: 2605.5689s
	iters: 200, epoch: 10

...

Epoch: 1 cost time: 19.645917654037476
Epoch: 1, Steps: 206 | Train Loss: 1.2526686 Vali Loss: 1.1838048 Test Loss: 1.3621862
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 1.3460201
	speed: 0.3276s/iter; left time: 3274.4456s
	iters: 200, epoch: 2 | loss: 1.0404321
	speed: 0.0554s/iter; left time: 548.1711s
Epoch: 2 cost time: 19.252249002456665
Epoch: 2, Steps: 206 | Train Loss: 1.2503389 Vali Loss: 1.1833932 Test Loss: 1.3623773
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 1.1080586
	speed: 0.3321s/iter; left time: 3250.8430s
	iters: 200, epoch: 3 | loss: 1.2268645
	speed: 0.0553s/iter; left time: 536.0946s
Epoch: 3 cost time: 19.109108448028564
Epoch: 3, Steps: 206 | Train Loss: 1.2499438 Vali Loss: 1.1840209 Test Loss: 1.3618450
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.000125
	iters: 100, epoch: 4 | loss: 1.2777907
	speed: 0.3282s/iter; left time: 3144.9771s
	iters: 200, epoch: 4 | loss: 1.2098114
	speed: 0.0550s/iter; left t

Epoch: 10 cost time: 19.958136796951294
Epoch: 10, Steps: 204 | Train Loss: 1.2492582 Vali Loss: 1.1869843 Test Loss: 1.3601803
EarlyStopping counter: 1 out of 5
Updating learning rate to 9.765625e-07
	iters: 100, epoch: 11 | loss: 1.2182776
	speed: 0.3714s/iter; left time: 2993.8253s
	iters: 200, epoch: 11 | loss: 1.1418313
	speed: 0.0816s/iter; left time: 649.5441s
Epoch: 11 cost time: 25.833982467651367
Epoch: 11, Steps: 204 | Train Loss: 1.2494248 Vali Loss: 1.1857711 Test Loss: 1.3601801
EarlyStopping counter: 2 out of 5
Updating learning rate to 4.8828125e-07
	iters: 100, epoch: 12 | loss: 1.1928020
	speed: 0.4386s/iter; left time: 3445.8054s
	iters: 200, epoch: 12 | loss: 1.1758101
	speed: 0.0758s/iter; left time: 588.0045s
Epoch: 12 cost time: 25.567073345184326
Epoch: 12, Steps: 204 | Train Loss: 1.2492584 Vali Loss: 1.1869843 Test Loss: 1.3601800
EarlyStopping counter: 3 out of 5
Updating learning rate to 2.44140625e-07
	iters: 100, epoch: 13 | loss: 1.2097988
	speed: 0.4365s

	iters: 100, epoch: 3 | loss: 1.1418583
	speed: 0.4291s/iter; left time: 4077.1205s
	iters: 200, epoch: 3 | loss: 1.2182988
	speed: 0.0685s/iter; left time: 644.2877s
Epoch: 3 cost time: 21.920875549316406
Epoch: 3, Steps: 200 | Train Loss: 1.2507080 Vali Loss: 1.1701096 Test Loss: 1.3549172
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.000125
	iters: 100, epoch: 4 | loss: 1.2692412
	speed: 0.3747s/iter; left time: 3484.7841s
	iters: 200, epoch: 4 | loss: 1.2564930
	speed: 0.0711s/iter; left time: 654.4168s
Epoch: 4 cost time: 24.17754030227661
Epoch: 4, Steps: 200 | Train Loss: 1.2505799 Vali Loss: 1.1708080 Test Loss: 1.3549076
EarlyStopping counter: 2 out of 5
Updating learning rate to 6.25e-05
	iters: 100, epoch: 5 | loss: 1.2479906
	speed: 0.3930s/iter; left time: 3576.8410s
	iters: 200, epoch: 5 | loss: 1.2692292
	speed: 0.0746s/iter; left time: 671.4797s
Epoch: 5 cost time: 23.653388738632202
Epoch: 5, Steps: 200 | Train Loss: 1.2506358 Vali Loss: 1.1708045 Test 

Epoch: 3, Steps: 204 | Train Loss: 1.2518308 Vali Loss: 1.2538821 Test Loss: 1.2722517
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.000125
	iters: 100, epoch: 4 | loss: 1.3247528
	speed: 0.3426s/iter; left time: 3251.3951s
	iters: 200, epoch: 4 | loss: 1.1631510
	speed: 0.0573s/iter; left time: 538.2481s
Epoch: 4 cost time: 20.41754698753357
Epoch: 4, Steps: 204 | Train Loss: 1.2516593 Vali Loss: 1.2541685 Test Loss: 1.2722149
EarlyStopping counter: 2 out of 5
Updating learning rate to 6.25e-05
	iters: 100, epoch: 5 | loss: 1.1900766
	speed: 0.3132s/iter; left time: 2907.8345s
	iters: 200, epoch: 5 | loss: 1.2349852
	speed: 0.0601s/iter; left time: 552.3234s
Epoch: 5 cost time: 19.51910376548767
Epoch: 5, Steps: 204 | Train Loss: 1.2521745 Vali Loss: 1.2515863 Test Loss: 1.2721981
Updating learning rate to 3.125e-05
	iters: 100, epoch: 6 | loss: 1.2439302
	speed: 0.3323s/iter; left time: 3017.8695s
	iters: 200, epoch: 6 | loss: 1.2708809
	speed: 0.0594s/iter; left time

Epoch: 11 cost time: 21.233067274093628
Epoch: 11, Steps: 202 | Train Loss: 1.2493415 Vali Loss: 1.2452654 Test Loss: 1.2739725
EarlyStopping counter: 3 out of 5
Updating learning rate to 4.8828125e-07
	iters: 100, epoch: 12 | loss: 1.1870648
	speed: 0.3489s/iter; left time: 2714.1449s
	iters: 200, epoch: 12 | loss: 1.2888125
	speed: 0.0628s/iter; left time: 481.9692s
Epoch: 12 cost time: 20.30795121192932
Epoch: 12, Steps: 202 | Train Loss: 1.2493407 Vali Loss: 1.2464163 Test Loss: 1.2739725
EarlyStopping counter: 4 out of 5
Updating learning rate to 2.44140625e-07
	iters: 100, epoch: 13 | loss: 1.1930286
	speed: 0.3454s/iter; left time: 2617.2281s
	iters: 200, epoch: 13 | loss: 1.1451429
	speed: 0.0660s/iter; left time: 493.5053s
Epoch: 13 cost time: 20.98323678970337
Epoch: 13, Steps: 202 | Train Loss: 1.2491039 Vali Loss: 1.2459559 Test Loss: 1.2739725
EarlyStopping counter: 5 out of 5
Early stopping
>>>>>>>testing : coffee_classification_Autoformer_custom_ftS_sl96_ll42_pl42_dm512_

In [ ]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

models = ['Autoformer', 'Informer', 'Transformer', 'DLinear', 'NLinear']
data_paths = ['gold_data_c.csv', 'silver_data_c.csv', 'crude_oil_data_c.csv', 'natural_gas_data_c.csv', 'corn_data_c.csv', 'coffee_data_c.csv']
label_lens = [14, 28, 42, 56]
pred_lens = [14, 28, 42, 56]
args = dotdict()

for data_path in data_paths:
    for pred_len in pred_lens:

        # basic config
        args.is_training = 1
        args.model_id = 'classification'
        args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

        # data loader
        args.data = 'custom'
        args.root_path = './dataset/commodity/'
        args.data_path = data_path
        args.features = 'M'
        args.target = 'OT'
        args.freq = 'd'
        args.checkpoints = './checkpoints/'

        # forecasting task
        args.seq_len = 96
        args.label_len = pred_len
        args.pred_len = pred_len

        # DLinear
        args.individual = False

        # Formers 
        args.embed_type = 0
        args.enc_in = 6
        args.dec_in = 6
        args.c_out = 6
        args.d_model = 512
        args.n_heads = 8
        args.e_layers = 2
        args.d_layers = 1
        args.d_ff = 2048
        args.moving_avg = 25
        args.factor = 3
        args.distil = True
        args.dropout = 0.05
        args.embed = 'timeF'
        args.do_predict = True

        # optimization
        args.num_workers = 10
        args.itr = 1
        args.train_epochs = 50
        args.patience = 5
        args.learning_rate = 0.0005
        args.batch_size = 8
        args.lradj = 'type1'
        args.des = 'Exp'

        # GPU
        args.gpu = 0
        args.devices = '0'
        args.use_gpu = True
        args.use_multi_gpu = False

        # print('Args in experiment:')
        # print(args)

        Exp = Exp_Main
        print(f'######################  {data_path}_{args.model}_{pred_len}_{pred_len}  ######################')
        if args.is_training:
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                    args.data_path.split("_")[0],
                    args.model_id,
                    args.model,
                    args.data,
                    args.features,
                    args.seq_len,
                    args.label_len,
                    args.pred_len,
                    args.d_model,
                    args.n_heads,
                    args.e_layers,
                    args.d_layers,
                    args.d_ff,
                    args.factor,
                    args.embed,
                    args.distil,
                    args.des, ii)

                exp = Exp(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                exp.test(setting)

                torch.cuda.empty_cache()


In [2]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='gold_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 14
args.pred_len = 14

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'gold_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 14, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [2]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : gold_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1650
val 239
test 489
	iters: 100, epoch: 1 | loss: 0.7150675
	speed: 0.2773s/iter; left time: 2828.8428s
	iters: 200, epoch: 1 | loss: 1.0009717
	speed: 0.0509s/iter; left time: 513.9020s
Epoch: 1 cost time: 33.884782552719116
Epoch: 1, Steps: 206 | Train Loss: 0.7102846 Vali Loss: 5.8019609 Test Loss: 8.5066147
Validation loss decreased (inf --> 5.801961).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.6824962
	speed: 0.4324s/iter; left time: 4321.6925s
	iters: 200, epoch: 2 | loss: 0.5746744
	speed: 0.0515s/iter; left time: 510.0212s
Epoch: 2 cost time: 33.46830439567566
Epoch: 2, Steps: 206 | Train Loss: 0.7020987 Vali Loss: 5.6443119 Test Loss: 8.4638128
Validation loss decreased (5.801961 --> 5.644312).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, e

In [1]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='silver_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 14
args.pred_len = 14

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 1
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'silver_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 14, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 1, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [2]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : silver_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1649
val 239
test 489
	iters: 100, epoch: 1 | loss: 0.4747304
	speed: 0.3702s/iter; left time: 39.6114s
	iters: 200, epoch: 1 | loss: 0.5830474
	speed: 0.0606s/iter; left time: 0.4242s
Epoch: 1 cost time: 44.29926419258118
Epoch: 1, Steps: 206 | Train Loss: 0.6441659 Vali Loss: 1.0920039 Test Loss: 1.9572040
Validation loss decreased (inf --> 1.092004).  Saving model ...
Updating learning rate to 0.0005
>>>>>>>testing : silver_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 489
( accuracy, f1: 0.696 & 0.726 
488, 488
40992, 40992
 accuracy, f1: 0.404 & 0.370 


In [4]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='crude_oil_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 14
args.pred_len = 14

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'crude_oil_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 14, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [6]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : crude_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1651
val 239
test 490
	iters: 100, epoch: 1 | loss: 0.7196439
	speed: 0.2879s/iter; left time: 2937.3040s
	iters: 200, epoch: 1 | loss: 0.7413711
	speed: 0.0513s/iter; left time: 517.7594s
Epoch: 1 cost time: 35.02819299697876
Epoch: 1, Steps: 206 | Train Loss: 0.8208497 Vali Loss: 1.4612168 Test Loss: 1.4606243
Validation loss decreased (inf --> 1.461217).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.8188974
	speed: 0.4571s/iter; left time: 4568.6189s
	iters: 200, epoch: 2 | loss: 0.8233802
	speed: 0.0512s/iter; left time: 506.9894s
Epoch: 2 cost time: 35.022679567337036
Epoch: 2, Steps: 206 | Train Loss: 0.8001896 Vali Loss: 1.4590780 Test Loss: 1.3978621
Validation loss decreased (1.461217 --> 1.459078).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, 

In [7]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='natural_gas_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 14
args.pred_len = 14

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'natural_gas_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 14, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [8]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : natural_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1651
val 239
test 490
	iters: 100, epoch: 1 | loss: 0.6901675
	speed: 0.2896s/iter; left time: 2954.0653s
	iters: 200, epoch: 1 | loss: 1.2720895
	speed: 0.0518s/iter; left time: 523.3331s
Epoch: 1 cost time: 35.24260234832764
Epoch: 1, Steps: 206 | Train Loss: 0.8955525 Vali Loss: 2.0291462 Test Loss: 7.2189631
Validation loss decreased (inf --> 2.029146).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.8486303
	speed: 0.4605s/iter; left time: 4602.9429s
	iters: 200, epoch: 2 | loss: 1.0460898
	speed: 0.0517s/iter; left time: 511.5081s
Epoch: 2 cost time: 35.08385729789734
Epoch: 2, Steps: 206 | Train Loss: 0.8843704 Vali Loss: 2.0378432 Test Loss: 7.2751784
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.6472185
	speed:

In [9]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='corn_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 14
args.pred_len = 14

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'corn_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 14, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [10]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : corn_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1648
val 239
test 489
	iters: 100, epoch: 1 | loss: 0.6302173
	speed: 0.2924s/iter; left time: 2982.4118s
	iters: 200, epoch: 1 | loss: 0.5111011
	speed: 0.0517s/iter; left time: 521.9231s
Epoch: 1 cost time: 35.497000217437744
Epoch: 1, Steps: 206 | Train Loss: 0.6896726 Vali Loss: 0.7556894 Test Loss: 3.0806372
Validation loss decreased (inf --> 0.755689).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.8323508
	speed: 0.4643s/iter; left time: 4641.0780s
	iters: 200, epoch: 2 | loss: 0.8922726
	speed: 0.0524s/iter; left time: 518.7365s
Epoch: 2 cost time: 35.49177956581116
Epoch: 2, Steps: 206 | Train Loss: 0.6776101 Vali Loss: 0.7755061 Test Loss: 3.1005499
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.4990257
	speed: 0

In [11]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='coffee_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 14
args.pred_len = 14

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'coffee_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 14, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [12]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : coffee_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1650
val 239
test 489
	iters: 100, epoch: 1 | loss: 0.6717560
	speed: 0.2922s/iter; left time: 2980.7278s
	iters: 200, epoch: 1 | loss: 0.5663242
	speed: 0.0519s/iter; left time: 524.6557s
Epoch: 1 cost time: 35.52903747558594
Epoch: 1, Steps: 206 | Train Loss: 0.8849214 Vali Loss: 0.9961210 Test Loss: 4.3159695
Validation loss decreased (inf --> 0.996121).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.7586984
	speed: 0.4639s/iter; left time: 4636.9572s
	iters: 200, epoch: 2 | loss: 0.6653163
	speed: 0.0517s/iter; left time: 511.8684s
Epoch: 2 cost time: 35.3178174495697
Epoch: 2, Steps: 206 | Train Loss: 0.8661423 Vali Loss: 1.0656055 Test Loss: 4.2815118
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.7586295
	speed: 0

In [13]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='gold_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 28
args.pred_len = 28

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'gold_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 28, 'pred_len': 28, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [14]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : gold_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1636
val 225
test 475
	iters: 100, epoch: 1 | loss: 0.6036149
	speed: 0.2953s/iter; left time: 2982.8015s
	iters: 200, epoch: 1 | loss: 0.7139179
	speed: 0.0552s/iter; left time: 552.1386s
Epoch: 1 cost time: 36.07482433319092
Epoch: 1, Steps: 204 | Train Loss: 0.6889766 Vali Loss: 5.7942133 Test Loss: 8.5532084
Validation loss decreased (inf --> 5.794213).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.5617464
	speed: 0.4698s/iter; left time: 4649.6308s
	iters: 200, epoch: 2 | loss: 0.4372019
	speed: 0.0552s/iter; left time: 540.8448s
Epoch: 2 cost time: 36.06253099441528
Epoch: 2, Steps: 204 | Train Loss: 0.6784700 Vali Loss: 5.7197633 Test Loss: 8.5285025
Validation loss decreased (5.794213 --> 5.719763).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, ep

In [17]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='silver_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 28
args.pred_len = 28

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'silver_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 28, 'pred_len': 28, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [18]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : silver_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1635
val 225
test 475
	iters: 100, epoch: 1 | loss: 0.7415410
	speed: 0.2968s/iter; left time: 2998.2327s
	iters: 200, epoch: 1 | loss: 0.5628698
	speed: 0.0555s/iter; left time: 554.9223s
Epoch: 1 cost time: 36.31732106208801
Epoch: 1, Steps: 204 | Train Loss: 0.6284080 Vali Loss: 1.1501398 Test Loss: 2.0240119
Validation loss decreased (inf --> 1.150140).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.5531747
	speed: 0.4730s/iter; left time: 4680.8213s
	iters: 200, epoch: 2 | loss: 0.5956247
	speed: 0.0553s/iter; left time: 542.0996s
Epoch: 2 cost time: 36.236055850982666
Epoch: 2, Steps: 204 | Train Loss: 0.6160459 Vali Loss: 1.1111615 Test Loss: 1.9955672
Validation loss decreased (1.150140 --> 1.111161).  Saving model ...
Updating learning rate to 0.00025
	iters: 100,

In [21]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='crude_oil_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 28
args.pred_len = 28

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'crude_oil_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 28, 'pred_len': 28, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [22]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : crude_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1637
val 225
test 476
	iters: 100, epoch: 1 | loss: 0.7813736
	speed: 0.2971s/iter; left time: 3001.0793s
	iters: 200, epoch: 1 | loss: 1.0524077
	speed: 0.0556s/iter; left time: 555.7866s
Epoch: 1 cost time: 36.35511612892151
Epoch: 1, Steps: 204 | Train Loss: 0.7973842 Vali Loss: 1.5683143 Test Loss: 1.4443076
Validation loss decreased (inf --> 1.568314).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.6355556
	speed: 0.4723s/iter; left time: 4674.0955s
	iters: 200, epoch: 2 | loss: 0.8990454
	speed: 0.0553s/iter; left time: 541.5876s
Epoch: 2 cost time: 36.19926643371582
Epoch: 2, Steps: 204 | Train Loss: 0.7897036 Vali Loss: 1.4759213 Test Loss: 1.3795062
Validation loss decreased (1.568314 --> 1.475921).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, e

In [24]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : gold_classification_Autoformer_custom_ftM_sl96_ll14_pl14_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1650
val 239
test 489
	iters: 100, epoch: 1 | loss: 0.7150675
	speed: 0.2930s/iter; left time: 2988.4866s
	iters: 200, epoch: 1 | loss: 1.0009717
	speed: 0.0520s/iter; left time: 524.7884s
Epoch: 1 cost time: 35.681819915771484
Epoch: 1, Steps: 206 | Train Loss: 0.7102846 Vali Loss: 5.8019609 Test Loss: 8.5066147
Validation loss decreased (inf --> 5.801961).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.6824962
	speed: 0.4716s/iter; left time: 4714.0353s
	iters: 200, epoch: 2 | loss: 0.5746744
	speed: 0.0517s/iter; left time: 511.1028s
Epoch: 2 cost time: 35.59051203727722
Epoch: 2, Steps: 206 | Train Loss: 0.7020987 Vali Loss: 5.6443119 Test Loss: 8.4638128
Validation loss decreased (5.801961 --> 5.644312).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, e

In [25]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='natural_gas_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 28
args.pred_len = 28

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'
# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'natural_gas_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 28, 'pred_len': 28, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [26]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : natural_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1637
val 225
test 476
	iters: 100, epoch: 1 | loss: 0.9038433
	speed: 0.2971s/iter; left time: 3000.8960s
	iters: 200, epoch: 1 | loss: 1.2418987
	speed: 0.0555s/iter; left time: 555.0736s
Epoch: 1 cost time: 36.35600709915161
Epoch: 1, Steps: 204 | Train Loss: 0.8919711 Vali Loss: 2.0839107 Test Loss: 7.1337161
Validation loss decreased (inf --> 2.083911).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 1.0575428
	speed: 0.4739s/iter; left time: 4690.3778s
	iters: 200, epoch: 2 | loss: 0.8329572
	speed: 0.0557s/iter; left time: 546.1489s
Epoch: 2 cost time: 36.44255757331848
Epoch: 2, Steps: 204 | Train Loss: 0.8874255 Vali Loss: 2.0596650 Test Loss: 7.1137242
Validation loss decreased (2.083911 --> 2.059665).  Saving model ...
Updating learning rate to 0.00025
	iters: 100,

In [5]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'DLinear' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='corn_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 28
args.pred_len = 28

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'DLinear', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'corn_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 28, 'pred_len': 28, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [30]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : corn_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1634
val 225
test 475
	iters: 100, epoch: 1 | loss: 0.5086083
	speed: 0.2962s/iter; left time: 2992.3232s
	iters: 200, epoch: 1 | loss: 0.5787496
	speed: 0.0554s/iter; left time: 554.1252s
Epoch: 1 cost time: 36.26370906829834
Epoch: 1, Steps: 204 | Train Loss: 0.6733489 Vali Loss: 0.7653006 Test Loss: 3.0912483
Validation loss decreased (inf --> 0.765301).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.6788849
	speed: 0.4722s/iter; left time: 4673.1789s
	iters: 200, epoch: 2 | loss: 0.8291518
	speed: 0.0553s/iter; left time: 541.7891s
Epoch: 2 cost time: 36.21080708503723
Epoch: 2, Steps: 204 | Train Loss: 0.6601443 Vali Loss: 0.7662546 Test Loss: 3.1142321
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.00025
	iters: 100, epoch: 3 | loss: 0.6309078
	speed: 0.

In [31]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')

# Formers 
parser.add_argument('--embed_type', type=int, default=0, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

# args = parser.parse_args()
args = dotdict()

# basic config
args.is_training = 1
args.model_id = 'classification'
args.model = 'Autoformer' #Autoformer/Informer/Transformer/DLinear/NLinear

# data loader
args.data = 'custom'
args.root_path = './dataset/commodity/'
args.data_path ='coffee_data_c.csv' 
args.features = 'M'
args.target = 'OT'
args.freq = 'd'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len = 96
args.label_len = 28
args.pred_len = 28

# DLinear
args.individual = False

# Formers 
args.embed_type = 0
args.enc_in = 6
args.dec_in = 6
args.c_out = 6
args.d_model = 512
args.n_heads = 8
args.e_layers = 2
args.d_layers = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 3
args.distil = True
args.dropout = 0.05
args.embed = 'timeF'
args.do_predict = True

# optimization
args.num_workers = 10
args.itr = 1
args.train_epochs = 50
args.patience = 5
args.learning_rate = 0.0005
args.batch_size = 8
args.lradj = 'type1'
args.des = 'Exp'

# GPU
args.gpu = 0
args.devices = '0'
args.use_gpu = True
args.use_multi_gpu = False

print('Args in experiment:')
print(args)

Exp = Exp_Main




Args in experiment:
{'is_training': 1, 'model_id': 'classification', 'model': 'Autoformer', 'data': 'custom', 'root_path': './dataset/commodity/', 'data_path': 'coffee_data_c.csv', 'features': 'M', 'target': 'OT', 'freq': 'd', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 28, 'pred_len': 28, 'individual': False, 'embed_type': 0, 'enc_in': 6, 'dec_in': 6, 'c_out': 6, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'do_predict': True, 'num_workers': 10, 'itr': 1, 'train_epochs': 50, 'patience': 5, 'learning_rate': 0.0005, 'batch_size': 8, 'lradj': 'type1', 'des': 'Exp', 'gpu': 0, 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False}


In [32]:
if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.data_path.split("_")[0],
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()

Use GPU: cuda:0
>>>>>>>start training : coffee_classification_Autoformer_custom_ftM_sl96_ll28_pl28_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 1636
val 225
test 475
	iters: 100, epoch: 1 | loss: 0.5853355
	speed: 0.2972s/iter; left time: 3002.2371s
	iters: 200, epoch: 1 | loss: 0.6268731
	speed: 0.0556s/iter; left time: 556.0614s
Epoch: 1 cost time: 36.37827157974243
Epoch: 1, Steps: 204 | Train Loss: 0.8894222 Vali Loss: 1.0232641 Test Loss: 4.3021717
Validation loss decreased (inf --> 1.023264).  Saving model ...
Updating learning rate to 0.0005
	iters: 100, epoch: 2 | loss: 0.8018863
	speed: 0.4739s/iter; left time: 4690.1961s
	iters: 200, epoch: 2 | loss: 0.7081355
	speed: 0.0558s/iter; left time: 547.1419s
Epoch: 2 cost time: 36.34145712852478
Epoch: 2, Steps: 204 | Train Loss: 0.8686871 Vali Loss: 0.9638785 Test Loss: 4.2979031
Validation loss decreased (1.023264 --> 0.963879).  Saving model ...
Updating learning rate to 0.00025
	iters: 100, 

In [ ]:
from win10toast import ToastNotifier
toaster = ToastNotifier()
toaster.show_toast("Hello World!!!",
"Python is 10 seconds awsm!",
icon_path="custom.ico",
duration=3)

In [ ]:
import pandas as pd
exp.args.root_path = './dataset/commodity/'
exp.args.data_path = 'gold_data.csv'

df = pd.read_csv(os.path.join(args.root_path, args.data_path))
df.head()
df.info()


In [ ]:
# df['date'] = pd.to_datetime(df['date'])
# # df.to_csv(os.path.join(args.root_path, args.data_path), index=False) 
# last_date = df['date'].max()
# new_rows = pd.DataFrame(columns=df.columns, index=range(1, 31))
# new_rows['date'] = pd.date_range(start=last_date + pd.DateOffset(1), periods=30)
# df = pd.concat([df, new_rows], sort=False)
# df.tail()



In [ ]:
if args.do_predict:
    print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    prediction=exp.predict(setting, True)
    prediction = np.load('./results/'+setting+'/real_prediction.npy')

    prediction.shape
    

In [ ]:
# from torch.utils.data import DataLoader
# import pandas as pd
# import numpy as np
# from datetime import datetime

# date_obj = '10/14/2022'
# #date_obj = datetime.strptime(date, '%m/%d/%Y')

# print(date_obj)

# def create_prediction_dataset(data, seq_len, pred_len, start_date):
#     start_index = data[data['date'] == start_date].index[0]
#     end_index = start_index + args.seq_len + args.pred_len
#     prediction_data = data.iloc[start_index:end_index]
#     return prediction_data

# # Create the prediction dataset

# pred_data = create_prediction_dataset(df, args.seq_len, args.pred_len, date_obj)
# pred_data.head()
# print(pred_data)

# pred_loader = DataLoader(pred_data, batch_size = args.batch_size, shuffle=False)

# predictions = exp.predict(setting, pred_loader, True)

In [ ]:
from data_provider.data_loader import Dataset_Pred
from torch.utils.data import DataLoader
from data_provider.data_loader import Dataset_Custom
Data = Dataset_Custom

freq = 'b'

Data = Dataset_Custom
timeenc = 0 if args.embed!='timeF' else 1
flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1

data_set = Data(
    root_path=args.root_path,
    data_path=args.data_path,
    flag=flag,
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    timeenc=timeenc,
    target=args.target, # HULL here
    freq=args.freq # 'h': hourly, 't':minutely
)
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)

print(len(data_set), len(data_loader))


In [ ]:
exp.predict(setting, data_loader)

In [ ]:
# When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

preds = np.load('./results/'+setting+'/pred.npy')
trues = np.load('./results/'+setting+'/true.npy')

# [samples, pred_len, dimensions]
print(preds.shape, trues.shape)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib
data_pred = np.load('./results/'+setting+'/real_prediction.npy')
data_pred = torch.from_numpy(data_pred).permute(0,2,1)

plt.figure()
print(data_pred.shape)
#预测OT
plt.plot(data_pred[-1,-1,:])
print(data_pred[-1,-1,:].shape)
plt.show()
plt.plot(data_pred[0,-1,:])
print(data_pred[0,-1,:].shape)
#plt.show()
# draw HUFL prediction
plt.plot(data_pred[0,0,:])
print(data_pred[-1,-1,:].shape)
plt.show()
'''
Ground Truth
'''
data_gt = np.load('./results/'+setting+'/true.npy')
data_gt = torch.from_numpy(data_gt).permute(0,2,1)

# OT
plt.plot(data_gt[-1,-1,:])
print(data_gt[-1,-1,:].shape)
plt.show()
plt.plot(data_gt[0,-1,:])
print(data_gt[0,-1,:].shape)
#plt.show()
# draw HUFL prediction
plt.plot(data_gt[0,0,:])
print(data_gt[-1,-1,:].shape)
plt.show()



In [7]:
setting = 'coffee_classification_Autoformer_custom_ftS_sl96_ll56_pl56_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0'

In [8]:
if (args.model == 'DLinear'):
    model_name = setting
    for root, dirs, files in os.walk("checkpoints"):
        for name in files:
            model_path = os.path.join(root, name)
            if model_name not in model_path:
                continue
            weights = torch.load(model_path,map_location=torch.device('cpu'))
            weights_list = {}
            weights_list['seasonal'] = weights['Linear_Seasonal.weight'].numpy()
            weights_list['trend'] = weights['Linear_Trend.weight'].numpy()

            #print(root)
            #print(root.split('/'))

            save_root = 'weights_plot/%s'%root.split('\\')[1] #changed from save_root = 'weights_plot/%s'%root.split('/')[1]
            if not os.path.exists('weights_plot'):
                os.mkdir('weights_plot')
            if not os.path.exists(save_root):
                os.mkdir(save_root)
        
            for w_name,weight in weights_list.items():
                fig,ax=plt.subplots()
                im=ax.imshow(weight,cmap='plasma_r')
                fig.colorbar(im,pad=0.03)
                plt.savefig(os.path.join(save_root,w_name + '.pdf'),dpi=500)
                plt.show()
                plt.close()

KeyError: 'Linear_Seasonal.weight'

In [4]:
setting

NameError: name 'setting' is not defined